## TEAK BASAL AREA AND DOMINANT HEIGHT COMPUTATION FOR 2022 MONITORING YEAR

### Import relavant libraries

In [1]:
import pyodbc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Database Connnection

In [2]:
db_path = r'C:\Users\GILBERT FG\Desktop\Readings\PSP_database.accdb'

conn = pyodbc.connect(
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    rf'DBQ={db_path};'
)

cursor = conn.cursor()

for table in cursor.tables(tableType='TABLE'):
    print(table.table_name)

Block register
Plot monitoring history
Plot-block relation
PSPs
Slope correction factors
Thinning history
TreeData


### Query TreeData table to select teak areas

In [3]:
# query database selecting TreeData table and filter for teak areas and 2022 monitoring year

query = r"SELECT * FROM TreeData WHERE AreaType = 'Teak' AND Plantations = 'Tain II' AND [Monitoring year] = 2022.0"

df = pd.read_sql(query, conn)

df.head()

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_10504\792079567.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,Species scientific name,H (m),DBH (cm),Merchantable height (m),REMARKS,Incorrect DBH,Incorrect H,Incorrect H / DBH,Exclude,Justification for exclusion
0,Tain II,Teak,2022.0,1.0,26,1240,1.0,Teak,Tectona grandis,1.25,0.0,None,,False,False,None,False,None
1,Tain II,Teak,2022.0,1.0,26,1240,2.0,Teak,Tectona grandis,2.00,2.5,None,,False,False,None,False,None
2,Tain II,Teak,2022.0,1.0,26,1240,3.0,Teak,Tectona grandis,3.25,4.2,None,,False,False,None,False,None
3,Tain II,Teak,2022.0,1.0,26,1240,4.0,Teak,Tectona grandis,5.50,5.8,None,,False,False,None,False,None
4,Tain II,Teak,2022.0,1.0,26,1240,5.0,Teak,Tectona grandis,4.00,5.5,None,,False,False,None,False,None


### Snapshot of the dataset

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15766 entries, 0 to 15765
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Plantations                  15766 non-null  object 
 1   AreaType                     15766 non-null  object 
 2   Monitoring year              15766 non-null  float64
 3   Monitoring month             15766 non-null  float64
 4   Monitoring day               15766 non-null  int64  
 5   PLOT                         15766 non-null  object 
 6   TREE NR                      15766 non-null  float64
 7   Tree SPECIES                 15766 non-null  object 
 8   Species scientific name      15766 non-null  object 
 9   H (m)                        15766 non-null  float64
 10  DBH (cm)                     15766 non-null  float64
 11  Merchantable height (m)      0 non-null      object 
 12  REMARKS                      14053 non-null  object 
 13  Incorrect DBH   

In [5]:
df.shape

(15766, 18)

### Check for Null values in the dataset

In [6]:
df.isna().sum()

Plantations                        0
AreaType                           0
Monitoring year                    0
Monitoring month                   0
Monitoring day                     0
PLOT                               0
TREE NR                            0
Tree SPECIES                       0
Species scientific name            0
H (m)                              0
DBH (cm)                           0
Merchantable height (m)        15766
REMARKS                         1713
Incorrect DBH                      0
Incorrect H                        0
Incorrect H / DBH              15766
Exclude                            0
Justification for exclusion    15766
dtype: int64

### Data Cleaning

In [7]:
df['REMARKS'].unique()

array(['', 'Top Broken', ' ', 'Climber', None], dtype=object)

In [8]:
df = df[~((df['REMARKS'] == 'Top Broken') | (df['REMARKS'] == 'Climber'))]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15714 entries, 0 to 15765
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Plantations                  15714 non-null  object 
 1   AreaType                     15714 non-null  object 
 2   Monitoring year              15714 non-null  float64
 3   Monitoring month             15714 non-null  float64
 4   Monitoring day               15714 non-null  int64  
 5   PLOT                         15714 non-null  object 
 6   TREE NR                      15714 non-null  float64
 7   Tree SPECIES                 15714 non-null  object 
 8   Species scientific name      15714 non-null  object 
 9   H (m)                        15714 non-null  float64
 10  DBH (cm)                     15714 non-null  float64
 11  Merchantable height (m)      0 non-null      object 
 12  REMARKS                      14001 non-null  object 
 13  Incorrect DBH        

### Statistics

In [10]:
df[['H (m)', 'DBH (cm)']].describe()

,H (m),DBH (cm)
count,15714.000000,15714.000000
mean,8.790481,10.431268
std,4.441176,6.092648
min,0.250000,0.000000
25%,4.750000,5.100000
50%,9.500000,10.100000
75%,12.500000,15.000000
max,21.000000,36.400000


#### Link Blocks to Plots

In [11]:
blk_query = r"SELECT * FROM [Plot-block relation] WHERE Plantations = 'Tain II' AND AreaType = 'Teak'"

blk_df = pd.read_sql(blk_query, conn)

blk_df.head()

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_10504\1406130451.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  blk_df = pd.read_sql(blk_query, conn)


,RelationID,Plantations,Plot no,AreaType,Deviating plant year,Block,Start date,End date
0,74,Tain II,1370,Teak,NaN,PF80,2020-05-01,9999-12-31 00:00:00
1,76,Tain II,1372,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
2,77,Tain II,1366,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
3,78,Tain II,1363,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
4,79,Tain II,1374,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00


In [12]:
blk_df = blk_df.rename(columns={'Plot no': 'PLOT'})
blk_df.head()

,RelationID,Plantations,PLOT,AreaType,Deviating plant year,Block,Start date,End date
0,74,Tain II,1370,Teak,NaN,PF80,2020-05-01,9999-12-31 00:00:00
1,76,Tain II,1372,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
2,77,Tain II,1366,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
3,78,Tain II,1363,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00
4,79,Tain II,1374,Teak,NaN,C17,2020-05-01,9999-12-31 00:00:00


In [13]:
blk_plot_df = blk_df[['PLOT', 'Block']]

In [14]:
df = pd.merge(df, blk_plot_df, on='PLOT')
df.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,Species scientific name,H (m),DBH (cm),Merchantable height (m),REMARKS,Incorrect DBH,Incorrect H,Incorrect H / DBH,Exclude,Justification for exclusion,Block
0,Tain II,Teak,2022.0,1.0,26,1240,1.0,Teak,Tectona grandis,1.25,0.0,None,,False,False,None,False,None,C10
1,Tain II,Teak,2022.0,1.0,26,1240,2.0,Teak,Tectona grandis,2.00,2.5,None,,False,False,None,False,None,C10
2,Tain II,Teak,2022.0,1.0,26,1240,3.0,Teak,Tectona grandis,3.25,4.2,None,,False,False,None,False,None,C10
3,Tain II,Teak,2022.0,1.0,26,1240,4.0,Teak,Tectona grandis,5.50,5.8,None,,False,False,None,False,None,C10
4,Tain II,Teak,2022.0,1.0,26,1240,5.0,Teak,Tectona grandis,4.00,5.5,None,,False,False,None,False,None,C10


In [15]:
df = df[['Block', 'PLOT', 'Monitoring year', 'TREE NR', 'H (m)', 'DBH (cm)']]

In [16]:
df.head()

,Block,PLOT,Monitoring year,TREE NR,H (m),DBH (cm)
0,C10,1240,2022.0,1.0,1.25,0.0
1,C10,1240,2022.0,2.0,2.00,2.5
2,C10,1240,2022.0,3.0,3.25,4.2
3,C10,1240,2022.0,4.0,5.50,5.8
4,C10,1240,2022.0,5.0,4.00,5.5


### Plot sizes

In [17]:
plt_query = r"SELECT * FROM [Plot monitoring history] WHERE Plantations = 'Tain II' AND AreaType = 'Teak'"

plt_df = pd.read_sql(plt_query, conn)

plt_df.head()

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_10504\2551688171.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  plt_df = pd.read_sql(plt_query, conn)


,MonitoringID,Plantations,Plot no,AreaType,Monitoring year,Monitoring Month,Monitoring day,Default plot size,Plot type,Monitoring team,Shrub cover (%),Herb cover (%),Invasive species cover (%),Erosion,Remarks,Excluded,Justification for exclusion,MDate
0,2154,Tain II,18,Teak,2018,1,22,729.0,Square,"Bentil, Ernest, Prince, Kwakye",NaN,NaN,None,False,None,False,None,2018-01-22
1,2155,Tain II,18,Teak,2019,1,24,729.0,Square,"Augustine,Enoch,Vida,Frank & Judith",NaN,NaN,None,False,None,False,None,2019-01-24
2,2156,Tain II,19,Teak,2018,2,13,729.0,Square,"Prince, Philp, Kwakye, Bentil, Danquah",NaN,NaN,None,False,None,False,None,2018-02-13
3,2157,Tain II,19,Teak,2019,1,23,729.0,Square,"Belinda,Kingsley,Prince,Stephen & Juliet",NaN,NaN,None,False,None,False,None,2019-01-23
4,2158,Tain II,20,Teak,2018,2,13,729.0,Square,"Prince, Philp, Kwakye, Bentil, Danquah",NaN,NaN,None,False,None,False,None,2018-02-13


#### Get Plot and sizes

In [18]:
plt_size_df = plt_df[['Plot no', 'Default plot size']]

In [19]:
plt_size_df = plt_size_df.rename(columns={'Plot no': 'PLOT'})
plt_size_df.head()

,PLOT,Default plot size
0,18,729.0
1,18,729.0
2,19,729.0
3,19,729.0
4,20,729.0


##### Convert plot sizes into hectares

In [20]:
plt_size_df['plot_size_ha'] = plt_size_df['Default plot size'] / 10000
plt_size_df.head()

,PLOT,Default plot size,plot_size_ha
0,18,729.0,0.0729
1,18,729.0,0.0729
2,19,729.0,0.0729
3,19,729.0,0.0729
4,20,729.0,0.0729


In [21]:
df = pd.merge(df, plt_size_df, on='PLOT')
df.head()

,Block,PLOT,Monitoring year,TREE NR,H (m),DBH (cm),Default plot size,plot_size_ha
0,C10,1240,2022.0,1.0,1.25,0.0,729.0,0.0729
1,C10,1240,2022.0,1.0,1.25,0.0,729.0,0.0729
2,C10,1240,2022.0,2.0,2.00,2.5,729.0,0.0729
3,C10,1240,2022.0,2.0,2.00,2.5,729.0,0.0729
4,C10,1240,2022.0,3.0,3.25,4.2,729.0,0.0729


### Compute for Stems Per Hectare (SPH) - Plot level

In [24]:
sph_df = df.groupby(['PLOT', 'plot_size_ha'])['TREE NR'].nunique().reset_index()
sph_df = sph_df.rename(columns={'TREE NR': 'tree_count'})
sph_df.head()

,PLOT,plot_size_ha,tree_count
0,1,0.0729,42
1,10,0.0729,44
2,11,0.0729,40
3,1141,0.0729,63
4,1142,0.0729,42


In [27]:
sph_df['SPH'] = sph_df['tree_count'] / sph_df['plot_size_ha']
sph_df['SPH'] = sph_df['SPH'].astype(int)
sph_df.head(50)

,PLOT,plot_size_ha,tree_count,SPH
0,1,0.0729,42,576
1,10,0.0729,44,603
2,11,0.0729,40,548
3,1141,0.0729,63,864
4,1142,0.0729,42,576
5,1143,0.0729,32,438
6,1144,0.0729,45,617
7,1145,0.0729,66,905
8,1146,0.0729,58,795
9,1147,0.0729,17,233


### Compute for Basal Area (BA) - Plot level

In [40]:
ba_df = df[['Block', 'PLOT', 'Monitoring year', 'plot_size_ha','H (m)', 'DBH (cm)']]
ba_df.head()

,Block,PLOT,Monitoring year,plot_size_ha,H (m),DBH (cm)
0,C10,1240,2022.0,0.0729,1.25,0.0
1,C10,1240,2022.0,0.0729,1.25,0.0
2,C10,1240,2022.0,0.0729,2.00,2.5
3,C10,1240,2022.0,0.0729,2.00,2.5
4,C10,1240,2022.0,0.0729,3.25,4.2


In [41]:
ba_df['ba_tree_m2'] = (np.pi * (ba_df['DBH (cm)']**2)/(4 * 10000))


C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_10504\2956318617.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ba_df['ba_tree_m2'] = (np.pi * (ba_df['DBH (cm)']**2)/(4 * 10000))


In [42]:
ba_df.head()

,Block,PLOT,Monitoring year,plot_size_ha,H (m),DBH (cm),ba_tree_m2
0,C10,1240,2022.0,0.0729,1.25,0.0,0.000000
1,C10,1240,2022.0,0.0729,1.25,0.0,0.000000
2,C10,1240,2022.0,0.0729,2.00,2.5,0.000491
3,C10,1240,2022.0,0.0729,2.00,2.5,0.000491
4,C10,1240,2022.0,0.0729,3.25,4.2,0.001385


In [43]:
ba_df = ba_df.groupby(['PLOT', 'plot_size_ha'])['ba_tree_m2'].sum().reset_index()

In [45]:
ba_df['ba_ha_m2'] = ba_df['ba_tree_m2'] / ba_df['plot_size_ha']
ba_df.head()

,PLOT,plot_size_ha,ba_tree_m2,ba_ha_m2
0,1,0.0729,6.205608,85.124944
1,10,0.0729,7.705345,105.697463
2,11,0.0729,6.512192,89.330476
3,1141,0.0729,0.215028,2.949628
4,1142,0.0729,0.105580,1.448279
